In [3]:
'''
신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모둘(module)로 구성되어 있습니다. torch.nn네임스페이스는 신경망을 구성하는데 필요한 모든 구성요소를 제공합니다.
pytorch의 모든 모듈을 nn.Module의 하위클래스(subclass)입니다. 신경망은 다른 모듈(계층:layer)로 구성된 모듈입니다.이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축
하고 관리할 수 있습니다.
'''

'\n신경망 모델 구성하기\n\n신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모둘(module)로 구성되어 있습니다. torch.nn네임스페이스는 신경망을 구성하는데 필요한 모든 구성요소를 제공합니다.\npytorch의 모든 모듈을 nn.Module의 하위클래스(subclass)입니다. 신경망은 다른 모듈(계층:layer)로 구성된 모듈입니다.이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축\n하고 관리할 수 있습니다.\n'

In [4]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
'''
학습을 위한 장치 얻기

가능한 경우 gpu와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다. torch.cuda를 사용할 수 있는지 확인하고 그렇지 않으면 cpu를 계속 사용합니다.
'''

'\n학습을 위한 장치 얻기\n\n가능한 경우 gpu와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다. torch.cuda를 사용할 수 있는지 확인하고 그렇지 않으면 cpu를 계속 사용합니다.\n'

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using{device}device")

Usingcpudevice


In [7]:
'''
클래스 정의하기

신경망 모델을 nn.Module의 하위 클래스로 정의하고, __init__에서 신경망 계층들을 초기화합니다.
nn.Module을 상속받은 모든 클래스는 forward메소드에 입력 데이터에 대한 연산들을 구현합니다.
'''

'\n클래스 정의하기\n\n신경망 모델을 nn.Module의 하위 클래스로 정의하고, __init__에서 신경망 계층들을 초기화합니다.\nnn.Module을 상속받은 모든 클래스는 forward메소드에 입력 데이터에 대한 연산들을 구현합니다.\n'

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() #https://dojang.io/mod/page/view.php?id=2386
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x=self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
'''
NeuralNetwork의 인스턴스를 생성하고 이를 device로 이동한 뒤 ,구조(structure)를 출력합니다.
'''

'\nNeuralNetwork의 인스턴스를 생성하고 이를 device로 이동한 뒤 ,구조(structure)를 출력합니다.\n'

In [26]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
'''
모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연살들과 함께 모델의 forward를 실행합니다. model.forward()를 직접 호출하지 마세요!

모델에 입력을 호출하면 각 분류에 대한 raw예측값이 있는 10-차원 텐서가 반환됩니다. 원시 예측값을 nn.Softmax모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.
'''

'\n모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연살들과 함께 모델의 forward를 실행합니다. model.forward()를 직접 호출하지 마세요!\n\n모델에 입력을 호출하면 각 분류에 대한 raw예측값이 있는 10-차원 텐서가 반환됩니다. 원시 예측값을 nn.Softmax모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.\n'

In [12]:
X=torch.rand(1,28,28,device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred=pred_probab.argmax(1)
print(f"Predicted class:{y_pred}")

Predicted class:tensor([0])


In [13]:
'''
모델 계층(layer)

FashionMNIST모델의 계층들을 살펴보겠습니다. 이를 설명하기 위해 28*28크기의 이미지 3개로 구성된 미니 배치를 가져와 신경망을 통과할 때 어떤 일이 발생하는지
알아보겠습니다.
'''

'\n모델 계층(layer)\n\nFashionMNIST모델의 계층들을 살펴보겠습니다. 이를 설명하기 위해 28*28크기의 이미지 3개로 구성된 미니 배치를 가져와 신경망을 통과할 때 어떤 일이 발생하는지\n알아보겠습니다.\n'

In [14]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [15]:
'''
nn.Flatten

nn.Flatten계층을 초기화하여 각 28*28의 2D 이미지를 784픽셀 값을 갖는 연속된 배열(1차원 배열)로 변환합니다.
(dim=0의 미니배치 차원은 유지됩니다.)
'''

'\nnn.Flatten\n\nnn.Flatten계층을 초기화하여 각 28*28의 2D 이미지를 784픽셀 값을 갖는 연속된 배열(1차원 배열)로 변환합니다.\n(dim=0의 미니배치 차원은 유지됩니다.)\n'

In [16]:
flatten=nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [17]:
'''
nn.Linear

선형계층은 저장된 가충치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈입니다.
'''

'\nnn.Linear\n\n선형계층은 저장된 가충치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈입니다.\n'

In [18]:
layer1 =nn.Linear(in_features =28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [19]:
'''
nn.ReLU
비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한관계(mapping)를 만듭니다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고,
신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.
'''

'\nnn.ReLU\n비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한관계(mapping)를 만듭니다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고,\n신경망이 다양한 현상을 학습할 수 있도록 돕습니다.\n\n이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.\n'

In [20]:
print(f"Before ReLU:{hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After TeLU:{hidden1}")

Before ReLU:tensor([[-0.2185, -0.1138, -0.6790, -0.5115, -0.8631,  0.2325,  0.9517,  0.3044,
          0.2685,  0.6746,  0.7679,  0.2908,  0.0106,  0.0237, -0.0542,  0.5555,
         -0.5401, -0.1466,  0.6095, -0.7999],
        [ 0.2957, -0.1370, -0.3205, -0.1306, -0.7693,  0.2092,  0.8975,  0.0020,
         -0.2480,  0.2270,  0.7642,  0.0053, -0.1574, -0.1059,  0.1803,  0.1119,
         -0.4305,  0.0728,  0.5055, -0.3280],
        [-0.0903, -0.2187, -0.3245, -0.3267, -1.0370, -0.1312,  0.6459,  0.0276,
          0.0029,  0.4838,  0.5443,  0.3014,  0.0171,  0.3080,  0.3398,  0.4000,
         -0.8667, -0.2691,  0.7344, -0.4569]], grad_fn=<AddmmBackward0>)


After TeLU:tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2325, 0.9517, 0.3044, 0.2685,
         0.6746, 0.7679, 0.2908, 0.0106, 0.0237, 0.0000, 0.5555, 0.0000, 0.0000,
         0.6095, 0.0000],
        [0.2957, 0.0000, 0.0000, 0.0000, 0.0000, 0.2092, 0.8975, 0.0020, 0.0000,
         0.2270, 0.7642, 0.0053, 0.0000, 0.0000, 0.1803

In [21]:
'''
nn.Sequential

nn.Sequential은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 순차 컨테이너를 사용하여 아래의 seq_modules
와 같은 신경망을 빠르게 만들 수 있습니다.
'''

'\nnn.Sequential\n\nnn.Sequential은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 순차 컨테이너를 사용하여 아래의 seq_modules\n와 같은 신경망을 빠르게 만들 수 있습니다.\n'

In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_img = torch.rand(3,28,28)
logits = seq_modules(input_img)
# input_img를 넣으면 첫번째로 flatten모듈에 입력되어 1차원 tenser으로 변형이 된 후 나온 결과를 layer1의 모듈에 입력하여 선형변환을 한다 이렇게 연쇄적으로 일어난다.

In [23]:
print(logits)

tensor([[-0.0596,  0.0126, -0.4781,  0.1232,  0.0458, -0.1076, -0.2078, -0.2385,
          0.2013,  0.0905],
        [ 0.0874,  0.1992, -0.5101,  0.0728,  0.0399, -0.0508, -0.2271, -0.3327,
          0.1810, -0.0185],
        [-0.1592,  0.2221, -0.5361,  0.2518,  0.0467, -0.1871, -0.2292, -0.4308,
          0.2244, -0.0887]], grad_fn=<AddmmBackward0>)


In [24]:
'''
nn.Softmax

신경망의 마지막 선형 계층은 nn.Softmax모듈에 전달될 ([-infty, infty] 범위의 원시 값(row value)인)logits를 반환합니다. logits는 모델의 각 분류(class)에 대한
예측 확률을 나타내도록[0,1]범위로 비례하여 조정됩니다. dim매개변ㅅ는 값의 함이 1이 되는 차원을 나타냅니다.
'''

'\nnn.Softmax\n\n신경망의 마지막 선형 계층은 nn.Softmax모듈에 전달될 ([-infty, infty] 범위의 원시 값(row value)인)logits를 반환합니다. logits는 모델의 각 분류(class)에 대한\n예측 확률을 나타내도록[0,1]범위로 비례하여 조정됩니다. dim매개변ㅅ는 값의 함이 1이 되는 차원을 나타냅니다.\n'

In [31]:
softmax=nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab[0].sum())
print(pred_probab[1].sum())
print(pred_probab[2].sum())
print(pred_probab)

tensor(1.0000, grad_fn=<SumBackward0>)
tensor(1.0000, grad_fn=<SumBackward0>)
tensor(1.0000, grad_fn=<SumBackward0>)
tensor([[0.0985, 0.1058, 0.0648, 0.1182, 0.1094, 0.0938, 0.0849, 0.0823, 0.1278,
         0.1144],
        [0.1128, 0.1262, 0.0621, 0.1112, 0.1076, 0.0983, 0.0824, 0.0741, 0.1239,
         0.1015],
        [0.0901, 0.1320, 0.0618, 0.1360, 0.1107, 0.0877, 0.0840, 0.0687, 0.1323,
         0.0967]], grad_fn=<SoftmaxBackward0>)


In [ ]:
'''
모델 매개변수

신경망 내부의 많은 계층들은 매개변수화 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. nn.Module을 상속하면 모델 객체 내부의 모든필드들이
자동으로 추적되며 모델의 parameters()및 named_parameters()메소드로 모든 매개변수에 접근할 수 있게 됩니다.

다음 예제에서는 각 매개변수들을 순회하며 매개변수의 크가와 값을 출력합니다.
'''

In [32]:
print("Model structure:",model,'\n\n')
for name, param in model.named_parameters():
    print(f"Layer:{name} | Size:{param.size()}|Values:{param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer:linear_relu_stack.0.weight | Size:torch.Size([512, 784])|Values:tensor([[-0.0058, -0.0069,  0.0069,  ..., -0.0045,  0.0050, -0.0350],
        [ 0.0333,  0.0267,  0.0006,  ...,  0.0178,  0.0027,  0.0050]],
       grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.0.bias | Size:torch.Size([512])|Values:tensor([-0.0316, -0.0199], grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.2.weight | Size:torch.Size([512, 512])|Values:tensor([[-0.0344,  0.0162,  0.0363,  ...,  0.0323,  0.0272,  0.0274],
        [-0.0011,  0.0337, -0.0190,  ...,  0.0305, -0.0149,  0.0122]],
       grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.2.bias | Size:torch.Size([512]